In [6]:
import os.path as osp

from tqdm import tqdm
import torch
from torch.nn import BCEWithLogitsLoss

import networkx as nx
import numpy as np
from torch_geometric.loader import DataLoader
from sklearn.metrics import roc_auc_score

from datasets import SEALDataset, SEALDatasetInMemory, SEALPredData
from models import DGCNN
from utils import read_temporary_graph_data, read_graph_from_edgefile

Super parameters:
 - `T`: split number of the time series.
 - `N`: number of predicted continuous time series.
 - `datasets`: the graph datasets, instance of `SEALDataset`.
 - `BS`: batch size.
 - `EPOCH`: number of epochs.
 - `LEARNING_RATE`: learning rate.

In [7]:
T = 400
N = 6
datasets = ['SuperUser',]
BS = 8192
EPOCH = 50
LEARN_RATE = 0.0001

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
np.random.seed(42)
torch.manual_seed(42)

In [4]:
def train(model, optimizer, criterion, data_loader, device='cuda'):
    model.train()

    total_loss = 0
    for data in data_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.batch)
        loss = criterion(out.view(-1), data.y.to(torch.float))
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * data.num_graphs

    return total_loss / len(data_loader.dataset)


@torch.no_grad()
def test(model, data_loader, device='cuda'):
    model.eval()

    y_pred, y_true = [], []
    for data in data_loader:
        data = data.to(device)
        logits = model(data.x, data.edge_index, data.batch)
        y_pred.append(logits.view(-1).cpu())
        y_true.append(data.y.view(-1).cpu().to(torch.float))

    return roc_auc_score(torch.cat(y_true), torch.cat(y_pred))


@torch.no_grad()
def predict(model, data_loader, threshold=0.5, device='cuda'):
    pred_edge_index = torch.zeros(0, 2).to(device)
    model.eval()
    
    for j, data in enumerate(data_loader):
        data = data.to(device)
        logits = model(data.x, data.edge_index, data.batch)
        logits = logits.view(-1).sigmoid()
        batch_mask = (logits > threshold).nonzero(as_tuple=True)[0]
        pred_edge_index = torch.cat((pred_edge_index, data.pred_edge[batch_mask]), dim=0)
    
    return pred_edge_index.type(torch.int).cpu()


def build_dataloader(dataset_name: str, t: int, additional_graphs=None, pred_from_add=True):
    n = 0 if additional_graphs is None else len(additional_graphs)
    print(f'\nLoading {dataset_name} Dataset with T={t} + {n}...')
    
    if n > 0 and pred_from_add:
        pred_graph = additional_graphs[-1]
    else:
        pred_graph = read_temporary_graph_data(
            f'data/SEALDataset/{dataset_name}/raw/{SEALDataset.info[dataset_name]["file"]}', 
            SEALDataset.info[dataset_name]["timespan"], t)[-1]
    assert pred_graph.number_of_nodes() > 30, f'The last temporal graph of {dataset_name} Dataset with T={t} is too small, consider to decrease T.'
    
    if dataset_name in ['WikiTalk', 'StackOverflow', 'SuperUser']:
        # For large dataset, we use SEALDataset just load last temporal graph data.
        # Cause splited data from last temporal graph's node feature size is much smaller than the whole last temporal graph.
        # So we should first load the whole last temporal graph data to get the max node feature size.
        pred_pos_data, pred_neg_data = SEALPredData.get_pos_neg_data(pred_graph)
        pred_data_list = SEALPredData.toSEAL_pred_datalist(pred_pos_data, pred_neg_data)
        print(f'Last temporal graph T={t} + {n} with {SEALPredData._max_z} node feature size.')
        
        # change the pred_idx range to choose the best training data.
        params = {'pred_idx': slice(-1, None), 'num_hops': 2, 'T': t, 'max_z': SEALPredData._max_z, 'additional_graphs': additional_graphs}
        train_dataset = SEALDataset('data/SEALDataset', dataset_name, 'train', **params)
        val_dataset = SEALDataset('data/SEALDataset', dataset_name, 'val', **params)
        test_dataset = SEALDataset('data/SEALDataset', dataset_name, 'test', **params)
        # Avoid splited data have more node feature size than the whole last temporal graph.
        if SEALPredData._max_z < train_dataset.num_features-1:
            pred_data_list = SEALPredData.toSEAL_pred_datalist(pred_pos_data, pred_neg_data, num_features=train_dataset.num_features)
            print(f'Update pred data node feature size to {train_dataset.num_features}.')
    else:
        # For small dataset, we use SEALDatasetInMemory to load all temporal graph data.
        params = {'num_hops': 2, 'T': t, 'additional_graphs': additional_graphs}
        train_dataset = SEALDatasetInMemory('data/SEALDataset', dataset_name, 'train', **params)
        val_dataset = SEALDatasetInMemory('data/SEALDataset', dataset_name, 'val', **params)
        test_dataset = SEALDatasetInMemory('data/SEALDataset', dataset_name, 'test', **params)
        # Obviously, the whole temporal graph's node feature size with the max node feature size.
        # So we use this num_features as the pred_data_list's node feature size.
        pred_pos_data, pred_neg_data = SEALPredData.get_pos_neg_data(pred_graph)
        pred_data_list = SEALPredData.toSEAL_pred_datalist(pred_pos_data, pred_neg_data, num_features=train_dataset.num_features)
        print(f'Last temporal graph T={t} + {n} with {SEALPredData._max_z} node feature size.')
    
    train_loader = DataLoader(train_dataset, batch_size=BS, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BS)
    test_loader = DataLoader(test_dataset, batch_size=BS)
    pred_loader = DataLoader(pred_data_list, batch_size=BS)
    return train_loader, val_loader, test_loader, pred_loader


def build_model(train_dataset, hidden_channels=32, num_layers=3):
    model = DGCNN(hidden_channels=hidden_channels, num_layers=num_layers, train_dataset=train_dataset).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARN_RATE)
    criterion = BCEWithLogitsLoss()
    return model, optimizer, criterion
    

def load_predict_graph(file_name: str):
    graph = read_graph_from_edgefile(file_name)
    # remove isolate node and self loop
    graph.remove_nodes_from(nx.isolates(graph))
    graph.remove_edges_from(nx.selfloop_edges(graph))
    return graph


def is_pred_edge_num_too_small(pred_edge_index, threshold=40):
    DG = nx.DiGraph()
    DG.add_edges_from(pred_edge_index.numpy())
    DG.remove_nodes_from(list(nx.isolates(DG)))
    DG.remove_edges_from(nx.selfloop_edges(DG))
    return DG.number_of_edges() < threshold


def is_pred_edge_num_too_many(pred_edge_index, threshold=7000):
    DG = nx.DiGraph()
    DG.add_edges_from(pred_edge_index.numpy())
    DG.remove_nodes_from(list(nx.isolates(DG)))
    DG.remove_edges_from(nx.selfloop_edges(DG))
    return DG.number_of_edges() > threshold


for dataset in datasets:
    predict_graphs = []
    t = T
    for n in range(1, N+1):
        # If current graph has been predicted, skip it.
        if osp.exists(f'data/SEALDataset/{dataset}/T{t}+{n}_pred_edge.pt'):
            print(f'{dataset}/T{t}+{n} has been predicted, just load.')
            predict_graphs.append(load_predict_graph(f'data/SEALDataset/{dataset}/T{t}+{n}_pred_edge.pt'))
            continue
        
        train_loader, val_loader, test_loader, pred_loader = build_dataloader(dataset, T, additional_graphs=predict_graphs)
        model, optimizer, criterion = build_model(train_loader.dataset)
        
        if osp.exists(f'data/models/SEAL_{dataset}_T{t}+{n-1}.pth'):
            model.load_state_dict(torch.load(f'data/models/SEAL_{dataset}_T{t}+{n-1}.pth'))
        else:
            print(f'\nTraining {dataset} Dataset with T={t}+{n-1}...')
            best_val_auc = test_auc = 0
            for epoch in tqdm(range(EPOCH)):
                loss = train(model, optimizer, criterion, train_loader)
                val_auc = test(model, val_loader)
                if val_auc > best_val_auc:
                    best_val_auc = val_auc
                    test_auc = test(model, test_loader)
                if epoch % 10 == 0 or epoch == EPOCH-1:
                    print(f'Epoch: [{epoch+1:02d}]/[{EPOCH:02d}], Loss: {loss:.4f}, Val: {val_auc:.4f}, Test: {test_auc:.4f}')
            print(f'Loss: {loss:.4f}, Val_best: {best_val_auc:.4f}, Test: {test_auc:.4f}')
            
            torch.save(model.state_dict(), f'data/models/SEAL_{dataset}_T{T}+{n-1}.pth')
        
        print(f'\nPredicting {dataset} Dataset with T={t} + {n}...')
        pred_edge_index = predict(model, pred_loader, threshold=0.5)
        if is_pred_edge_num_too_small(pred_edge_index):
            # Avoid no edge predicted, we will descend the threshold by 0.01 until we get some edges.
            for threshold in range(49, 0, -1):
                print(f'No edge predicted in {dataset} Dataset with T={t} + {n}, try threshold={threshold/100}...')
                pred_edge_index = predict(model, pred_loader, threshold=threshold/100)
                if not is_pred_edge_num_too_small(pred_edge_index):
                    break
        assert not is_pred_edge_num_too_small(pred_edge_index), f'No edge predicted in {dataset} Dataset with T={t} + {n}...'
        if is_pred_edge_num_too_many(pred_edge_index):
            # Avoid too many edges predicted, we will ascend the threshold by 0.01 until we get less edges.
            for threshold in range(51, 100, 1):
                print(f'Too many edges predicted in {dataset} Dataset with T={t} + {n}, try threshold={threshold/100}...')
                pred_edge_index = predict(model, pred_loader, threshold=threshold/100)
                if not is_pred_edge_num_too_many(pred_edge_index):
                    break
        assert not is_pred_edge_num_too_many(pred_edge_index), f'Too many edges predicted in {dataset} Dataset with T={t} + {n}...'
        torch.save(pred_edge_index, f'data/SEALDataset/{dataset}/T{t}+{n}_pred_edge.pt')
        # load current pred graph to predict_graphs
        print(f'load {dataset}/T{t}+{n}_pred_edge.pt to dataset')
        predict_graphs.append(load_predict_graph(f'data/SEALDataset/{dataset}/T{t}+{n}_pred_edge.pt'))

        torch.cuda.empty_cache()


Loading SuperUser Dataset with T=400 + 0...
Last temporal graph T=400 + 0 with 31 node feature size.

Training SuperUser Dataset with T=400+0...


  2%|▏         | 1/50 [00:04<03:29,  4.27s/it]

Epoch: [01]/[50], Loss: 0.6694, Val: 0.5032, Test: 0.4930


 22%|██▏       | 11/50 [00:06<00:10,  3.61it/s]

Epoch: [11]/[50], Loss: 0.6666, Val: 0.5967, Test: 0.5771


...

Epoch: [31]/[50], Loss: 0.6901, Val: 0.4985, Test: 0.4896


 82%|████████▏ | 41/50 [00:18<00:03,  2.46it/s]

Epoch: [41]/[50], Loss: 0.6838, Val: 0.5037, Test: 0.4975


100%|██████████| 50/50 [00:22<00:00,  2.18it/s]

Epoch: [50]/[50], Loss: 0.6773, Val: 0.5081, Test: 0.5021
Loss: 0.6773, Val_best: 0.5081, Test: 0.5021

Predicting SuperUser Dataset with T=400 + 6...
load SuperUser/T400+6_pred_edge.pt to dataset


In [5]:
predict_graphs

In [ ]:
import networkx as nx
from matplotlib import pyplot as plt

for g in predict_graphs:    
    options = {
        'node_color': '#1f78b4',
        'node_size': 500,
        'width': 3,
        'arrowstyle': '-|>',
        'arrowsize': 12,
    }
    ax = plt.figure(figsize=(10, 10), dpi=100)
    nx.draw_networkx(g, pos=nx.spring_layout(g), **options)
    plt.show()